In [1]:
import locale
locale.getpreferredenoding = lambda: "UTF:8"

In [2]:
!nvidia-smi

Sat Apr  1 05:58:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!git clone https://github.com/tloen/alpaca-lora.git

fatal: destination path 'alpaca-lora' already exists and is not an empty directory.


In [4]:
%cd alpaca-lora

/content/alpaca-lora


In [5]:
!pip install -u pip
!pip install -r requirements.txt
!pip install torch==2.0.0


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-7cf4_x1m
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-7cf4_x1m
  Resolved https://github.com/huggingface/peft.git to commit 11edb618c32065083b89918f26ba6f43c4c1804b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-tlo52bll
  Running command git clone --fi

In [6]:
import torch
from peft import PeftModel
import transformers
import textwrap
from transformers import LlamaTokenizer, GenerationConfig, LlamaForCausalLM
from transformers.generation.utils import GreedySearchDecoderOnlyOutput


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-3syilv8l7e00i --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=true'), PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=h

In [7]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [8]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [12]:
model = LlamaForCausalLM.from_pretrained("decapoda-research/llama-7b-hf", load_in_8bit=True, device_map="auto")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [13]:
model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b", torch_dtype=torch.float16)

In [14]:
    # unwind broken decapoda-research config
    model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
    model.config.bos_token_id = 1
    model.config.eos_token_id = 2

In [15]:
model = model.eval()
model = torch.compile(model)

In [27]:
PROMPT_TEMPLATE = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request

### Instruction:
[Instruction]

### Response:
"""

CLASSIFICATION_TEMPLATE = f"""
Below is the complete content of a document. Can you identify which category it is likely to be belonging to?.
The possible categories are given below.

1. Insurance claim request
2. Leave of absense letter 
3. Invitation letter

Write a response which identifies the document to any one of the above classes

### Content:

From,
Sherlock Holmes
221B Baker Street

Date: 01/04/2023

To,
General Insurance Company
Lombard Street London
United Kingdom

Dear sir/madam;

Re: Policy No - P-123-456-789

I am writing this letter in response to your letter on 30/03/2023, offering £400. to fix my claim. I don’t think that this is an appropriate sum.

I request you to check out my case. If I do not get a reasonable offer, I will have no alternative but to refer my case to the Financial Ombudsman Service.


I will review forward to get your offer within the following 14 days.

Yours sincerely/faithfully;

Sign and print your name.

### Document Classs:


"""

QUERY_TEMPLATE = f"""

Based on the tabular contents provided below, can you identify the total premium paid. Output the total premium paid only.

### Content:

### Response:

 """


def create_prompt(instruction: str) -> str:
    return PROMPT_TEMPLATE.replace("[Instruction]", instruction)


In [23]:
prompt = CLASSIFICATION_TEMPLATE
encoding = tokenizer(prompt, return_tensors="pt" )


In [24]:
input_ids = encoding["input_ids"].to(DEVICE)

In [19]:
generation_config = GenerationConfig(temperature=0.1, top_p=0.75, repetition_penalty=1.1)

In [25]:
with torch.inference_mode():
    output = model.generate(input_ids=input_ids, 
                            generation_config=generation_config, 
                            return_dict_in_generate=True, 
                            output_scores=True,
                            max_new_tokens=256)

In [26]:
response = tokenizer.decode(output.sequences[0])

print(response.split("### Document Classs:")[-1].strip())

Insurance Claim Request
